In [299]:
import pandas as pd
import os
import re

In [300]:
current_path = os.getcwd()
#我需要的資料夾
folders = []
for folder in os.listdir(current_path):
    #我要抓資料夾
    if '.' not in folder:
        folders.append(folder)

In [301]:
folders

['2020年交易監控月報', '2021年交易監控月報', '202201', '202202']

In [302]:
for folder in folders:
    print(re.search('\d+',folder)[0])

2020
2021
202201
202202


In [303]:
re.match(r'appl[e|b]','applc')

In [304]:
#2020有問題，還須處理


#資料亂到把我醜哭了 ^U^
#2020/12以前的是舊系統
dict_by_month = {}
for folder in folders:
    print('-'*50)
    print(folder)
    
    #判斷是不是整年的資料
    #2021年的資料
    #下面會有日報的資料，分成關聯非關聯
    if '2020' not in folder and '年' in folder:
        #particular_folder 是d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2020年交易監控月報
        particular_folder = os.getcwd() + '\\' + folder
        #找partiuclar_folder底下的資料夾
        months_folders = os.listdir(particular_folder)
        for month_folder in months_folders:
            month_list = []
            #進到每個月的資料夾
            particular_path = particular_folder + '\\' + month_folder
            files = os.listdir(particular_path)
            for file in files:
                #找資料夾下面的.csv or .xlsx檔案
                if re.match(r'TXT_[M|D]',file):
                    month_list.append(particular_path + '\\' + file)

                elif '日報-單一' in file and '.' not in file:
                    日報_folder = particular_path + '\\' + file
                    for 日報_file in os.listdir(日報_folder):
                        if re.match(r'TXT_[M|D]',日報_file):
                            month_list.append(日報_folder + '\\' + 日報_file)
                else:
                    pass
            dict_by_month[month_folder] = month_list

    #這邊就是2022年每月的了
    #下面會有日報的資料，分成關聯非關聯
    #跟月報
    #和2021得差別在於資料層數，所以當2022過完後，進行檔案整理，資料層數增加，這邊的判斷式就需要修改
    elif '2022' in folder:
        #particular_folder 是d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\202201
        particular_folder = os.getcwd() + '\\' + folder
        #找partiuclar_folder底下的資料夾
        months_folders = os.listdir(particular_folder)
        month_list = []
        for month_folder in months_folders:
            if re.match(r'TXT_[M|D]',month_folder):
                month_list.append(particular_folder + '\\' + month_folder)
            elif '日報-單一' in month_folder:
                日報_folder = particular_folder + '\\' + month_folder
                for 日報_file in os.listdir(日報_folder):
                    if re.match(r'TXT_[M|D]',日報_file):
                        month_list.append(日報_folder + '\\' + 日報_file)
            else:
                pass
            dict_by_month[folder] = month_list

    elif '2020' in folder:
        #12月以前是舊資料
        #particular_folder 是d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2020年交易監控月報
        particular_folder = os.getcwd() + '\\' + folder
        #找partiuclar_folder底下的資料夾
        months_folders = os.listdir(particular_folder)
        for month_folder in months_folders:
            month_list = []
            if '12' in month_folder:
                #進到每個月的資料夾
                particular_path = particular_folder + '\\' + month_folder
                files = os.listdir(particular_path)
                for file in files:
                    #找資料夾下面的.csv or .xlsx檔案
                    if re.match(r'TXT_[M|D]',file) and re.search(r'.xlsx\Z',file):
                        month_list.append(particular_path + '\\' + file)
                    
                    #DB單一日報
                    elif 'DB' in file and '.' not in file:
                        日報_folder = particular_path + '\\' + file
                        for 日報_file in os.listdir(日報_folder):
                            if re.match(r'\d+',日報_file) and '.xlsx' in 日報_file:
                                month_list.append(日報_folder + '\\' + 日報_file)
                    else:
                        pass
            else: 
                #進到每個月的資料夾
                particular_path = particular_folder + '\\' + month_folder
                files = os.listdir(particular_path)
                for file in files:
                    #找資料夾下面的.csv or .xlsx檔案
                    if re.search(r'#\d+[R|(R)]',file) and '.xlsx' in file:
                        month_list.append(particular_path + '\\' + file)

                    #DB單一
                    elif 'DB' in file and '.' not in file:
                        DB_folder = particular_path + '\\' + file
                        for DB_file in os.listdir(DB_folder):
                            if re.match(r'\d+-#\d+',DB_file) and '.xlsx' in DB_file:
                                month_list.append(DB_folder + '\\' + DB_file)
                            elif '.' not in DB_file:
                                DB_next_folder = DB_folder + '\\' + DB_file
                                for final_file in os.listdir(DB_next_folder):
                                    if re.search(r'#\d+',final_file) and '.xlsx' in final_file:
                                        month_list.append(DB_next_folder + '\\' + final_file)
            dict_by_month[month_folder] = month_list

--------------------------------------------------
2020年交易監控月報
--------------------------------------------------
2021年交易監控月報
--------------------------------------------------
202201
--------------------------------------------------
202202


In [305]:
new_dict = {}
for months, file_lists in dict_by_month.items():
    new_dict[months] = [file for file in file_lists if 'reordered' not in file and '交易監控統計表' not in file]

In [306]:
#把內容有  本檢核期間並無符合條件之案例  的資料刪掉
final_dict = {}
for months, file_lists in new_dict.items():
    #先看檔名沒有'$'在看內容有沒有 '本檢核期間並無符合條件之案例'
    final_dict[months] = [file for file in file_lists if '$' not in file and '本檢核期間並無符合條件之案例' not in pd.read_excel(file).values]
        

C:\Users\andyhs\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [307]:
final_dict.keys()

dict_keys(['202001', '202002', '202003', '202004', '202005', '202006', '202007', '202008', '202009', '202010', '202011', '202012', '202101', '202102', '202103', '202104', '202105', '202106', '202107', '202108', '202109', '202110', '202111', '202112', '202201', '202202'])

In [292]:
子集_2022 = [key for key in final_dict.keys() if '2022' in key]
dict_2022 = {}
確認用df_2022 = []
for month in 子集_2022:
    單月總計 = pd.DataFrame({})
    for x in final_dict[month]:
        each_file = pd.read_excel(x)
        if '案件號' in each_file.columns:
            print('')
        else:
            for index,value in enumerate(each_file.iloc[:,0]):
                if value == '案件號' or value == '歸戶編號':
                    break
            print(index)
            each_file = pd.read_excel(x,skiprows = index+1)
        
        
        if len(each_file) != 0:
            each_file['source'] = re.search(r'#\d+',x)[0]
            #因為#17 #19是契約代號，not my business
            if re.search(r'#\d+',x)[0] != '#17' and re.search(r'#\d+',x)[0] != '#19':
                確認用df_2022.append([re.search(r'#\d+',x)[0],len(each_file),month])
                print(len(each_file))
                print(len(each_file['戶號']))
                if len(each_file) != len(each_file['戶號']):
                    print(x)
                單月總計 = pd.concat([單月總計,each_file['戶號']],axis=0)
            
    dict_2022[month] = 單月總計

3
3
3
3
2
2
3
466
466
3
33
33
3
3
3
3
3
22
22
3
3
160
160
3
3
3
3
3
41
41
3
42
42
3
11
11
3
271
271
3
15
15
3
308
308
3
11
11
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
3
1
1
3
3
1
1
3
3
2
2
3
3
3
3
3
3
2
2
3
2
2
3
3
3
3
1
1
3
3
1
1
3
2
2
3
1
1
3
1
1
3
3
4
4
3
3
2
2
3
3
3
3
3
2
2
3
2
2
3
3
3
3
1
1
3
3
1
1
3
2
2
3
3
3
3
1
1
3
3
1
1
3
3
2
2
3
3
3
2
2
3
3
3
3
3
3
2
2
3
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
3
574
574
3
34
34
3
3
3
3
3
8
8
3
3
123
123
3
3
3
3
3
88
88
3
8
8
3
19
19
3
321
321
3
14
14
3
165
165
3
5
5
3
3
3
3
1
1
3
3
3
1
1
3
3
3
3
1
1
3
3
3
3
1
1
3
3
3
3
12
12
3
1
1
3
1
1
3
1
1
3
1
1
3
1
1
3
1
1
3
3
1
1
3
1
1
3
3
3
2
2
3
3
2
2
3
3
3
3
3
3
3
2
2
3
3
3
3
3
3
3
3
1
1
3
3
1
1
3
1
1
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


In [308]:
子集_2021 = [key for key in final_dict.keys() if '2021' in key]
dict_2021 = {}
確認用df_2021 = []
for month in 子集_2021:
    print(month)
    單月總計 = pd.DataFrame({})
    for x in final_dict[month]:
        each_file = pd.read_excel(x)
        if '案件號' in each_file.columns:
            print('')
        else:
            for index,value in enumerate(each_file.iloc[:,0]):
                if value == '案件號' or value == '歸戶編號':
                    break
            print(index)
            each_file = pd.read_excel(x,skiprows = index+1)
        
        
        if len(each_file) != 0:
            each_file['source'] = re.search(r'#\d+',x)[0]
            #因為#17 #19是契約代號，not my business
            if re.search(r'#\d+',x)[0] != '#17' and re.search(r'#\d+',x)[0] != '#19':
                確認用df_2021.append([re.search(r'#\d+',x)[0],len(each_file),month])
                print(len(each_file))
                print(len(each_file['戶號']))
                if len(each_file) != len(each_file['戶號']):
                    print(x)
                單月總計 = pd.concat([單月總計,each_file['戶號']],axis=0)
    print('-'*50)
            
    dict_2021[month] = 單月總計

202101
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202101\202101日報-單一\TXT_D_#11_20201231_1.xlsx
3
1
1
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202101\202101日報-單一\TXT_D_#11_20210104_1.xlsx
3
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202101\202101日報-單一\TXT_D_#11_20210105_1.xlsx
3
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202101\202101日報-單一\TXT_D_#11_20210106_1.xlsx
3
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202101\202101日報-單一\TXT_D_#11_20210107_1.xlsx
3
1
1
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202101\202101日報-單一\TXT_D_#11_20210108_1.xlsx
3
3
3
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202101\202101日報-單一\TXT_D_#11_20210111_1.xlsx
3
1
1
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202101\202101日報-單一\TXT_D_#11_20210112_1.xlsx
3
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202101\202101日報-單一\TXT_D_#11_20210113_1.xlsx
3
2
2
d:\My Documents\andyhs\桌面\An

C:\Users\andyhs\AppData\Local\Temp/ipykernel_25652/1023927266.py:17: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  each_file = pd.read_excel(x,skiprows = index+1)


3
3
3
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202102\日報-單一\TXT_D_#14_20210226_1.xlsx
3
4
4
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202102\日報-單一\TXT_D_#17_20210201_1.xlsx
3
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202102\日報-單一\TXT_D_#17_20210202_1.xlsx
3
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202102\日報-單一\TXT_D_#17_20210203_1.xlsx
3
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202102\日報-單一\TXT_D_#17_20210204_1.xlsx
3
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202102\日報-單一\TXT_D_#17_20210205_1.xlsx
3
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202102\日報-單一\TXT_D_#17_20210206_1.xlsx
3
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202102\日報-單一\TXT_D_#17_20210207_1.xlsx
3
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202102\日報-單一\TXT_D_#17_20210208_1.xlsx
3
d:\My Documents\andyhs\桌面\Andy\Fontaine TXT分析\2021年交易監控月報\202102\日報-單一\TXT_D_#17_20210209_1.xlsx
3


In [314]:
dict_2021['202102']

,0
0,6001.0
1,160567.0
2,6001.0
3,160567.0
4,218436.0
...,...
2,94533.0
0,306304.0
1,306751.0
2,310150.0


In [315]:
c = pd.DataFrame(確認用df_2021)

In [316]:
c[c[2]=='202102'][1].sum()

3116